In [18]:
import pandas as pd

df = pd.read_excel("星云二号方案-浮动基准1号.xlsx", sheet_name='净值图')
df = df[df.columns[0:6]].dropna()
df['交易日'] = pd.to_datetime(df['交易日']).dt.date
df['基准1超额收益率'] = df['相对基准1超额'] - 1
df['基准2超额收益率'] = df['相对基准2超额'] - 1
df

,交易日,组合净值,基准1净值,相对基准1超额,基准2净值,相对基准2超额,基准1超额收益率,基准2超额收益率
0,2016-01-04,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
1,2016-01-05,0.994901,0.999452,0.995449,0.993845,1.001056,-0.004551,0.001056
2,2016-01-06,1.002406,1.006182,0.996224,1.001660,1.000746,-0.003776,0.000746
3,2016-01-07,0.975124,0.981018,0.994106,0.972540,1.002583,-0.005894,0.002583
4,2016-01-08,0.978568,0.987166,0.991401,0.975028,1.003540,-0.008599,0.003540
...,...,...,...,...,...,...,...,...
1810,2023-06-14,1.658545,1.240116,1.418429,1.322738,1.335807,0.418429,0.335807
1811,2023-06-15,1.666251,1.245785,1.420466,1.329461,1.336791,0.420466,0.336791
1812,2023-06-16,1.671573,1.249521,1.422053,1.334065,1.337508,0.422053,0.337508
1813,2023-06-19,1.670753,1.246776,1.423977,1.332740,1.338013,0.423977,0.338013


参考链接：

1. [Pyecharts之折线图与柱状图组合绘制](https://blog.csdn.net/weixin_48488448/article/details/114161235)
2. [pyecharts重叠两种图（柱状+折线）](https://blog.csdn.net/w15059001146/article/details/106148652)
3. [Grid - Grid_multi_yaxis](https://gallery.pyecharts.org/#/Grid/grid_multi_yaxis)
4. [pyechart实现曲线图与柱形图组合](https://zhuanlan.zhihu.com/p/367180023)

In [19]:
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Bar, Grid, Line
from pyecharts import options as opts

line = Line() #创建Line实例

# 设置折线1
line.add_xaxis(df['交易日'].tolist())
line.add_yaxis('组合净值', df['组合净值'].tolist(), label_opts=opts.LabelOpts(is_show=False), color='#88c4e8')
line.add_yaxis('基准1净值', df['基准1净值'].tolist(), label_opts=opts.LabelOpts(is_show=False), color='#ea9c9d')
line.add_yaxis('基准2净值', df['基准2净值'].tolist(), label_opts=opts.LabelOpts(is_show=False), color='#93cc82')

# 设置y轴1最大值、最小值和间隔
line.set_global_opts(
    yaxis_opts=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(
            formatter=JsCode(
                "function(value){return value.toFixed(1)}"
            )
        ),
        splitline_opts=opts.SplitLineOpts(is_show=True),
        name="收益",
        min_=0.9,  # 左侧y轴最小值
        max_=1.9,  # 左侧y轴最大值
        interval=0.1  # 左侧y轴间隔
    ),
    datazoom_opts=[
        opts.DataZoomOpts(
            is_show=True, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
            ),
            opts.DataZoomOpts(
                is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                ),
                ],
    tooltip_opts=opts.TooltipOpts(
        trigger="axis",
        axis_pointer_type="cross",
        background_color="rgba(245, 245, 245, 0.8)",
        border_width=1,
        border_color="#ccc",
        textstyle_opts=opts.TextStyleOpts(color="#000"),
        ),
    axispointer_opts=opts.AxisPointerOpts(
        is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
        ),
)

bar = Bar()

bar.add_yaxis('相对基准1超额', df['基准1超额收益率'].tolist(), yaxis_index=1, label_opts=opts.LabelOpts(is_show=False), color='#F5F5DC')
bar.add_yaxis('相对基准2超额', df['基准2超额收益率'].tolist(), yaxis_index=1, label_opts=opts.LabelOpts(is_show=False), color='#EEE8AA')

line = line.overlap(bar)
# 设置右侧y轴，并按需添加is_inverse=True参数
line.extend_axis(
    yaxis=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(
            formatter=JsCode(
                "function(value){return value.toFixed(1)}"
            )
        ),
        name='超额',
        min_=0,
        max_=1,
        interval=0.1,
        position='right',
        # is_inverse=True
    )
)

#line = line.overlap(bar)
# 隐藏数据系列标签，使图表更加简洁
line.set_series_opts(label_opts=opts.LabelOpts(is_show=False))

line.render('模拟组合历史数据.html')

'c:\\Users\\siruo\\Downloads\\模拟组合历史回测\\模拟组合历史数据.html'

In [20]:
bar = (
    Bar()
    .add_xaxis(df['交易日'].tolist())
    .add_yaxis(
        '相对基准1超额',
        df['基准1超额收益率'].tolist(),
        yaxis_index=0,
        label_opts=opts.LabelOpts(is_show=False),
        color='#F5F5DC')
    .add_yaxis(
        '相对基准2超额',
        df['基准2超额收益率'].tolist(),
        yaxis_index=1,
        label_opts=opts.LabelOpts(is_show=False),
        color='#EEE8AA')
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="超额1",
            type_="value",
            min_=0,
            max_=1,
            position="right",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="#d14a61")
            ),
            #axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
        )
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            type_="value",
            name="收益",
            min_=0.9,
            max_=1.9,
            position="left",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="#675bba")
            ),
            #axislabel_opts=opts.LabelOpts(formatter="{value} °C"),
            splitline_opts=opts.SplitLineOpts(
                is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=1)
            ),
        )
    )
    .set_global_opts(
        yaxis_opts=opts.AxisOpts(
            name="超额2",
            min_=0,
            max_=1,
            position="right",
            offset=80,
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="#5793f3")
            ),
            #axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
        ),
        #title_opts=opts.TitleOpts(title="Grid-多 Y 轴示例"),
        # tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        datazoom_opts=[
            opts.DataZoomOpts(
                is_show=True, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
                ),
                opts.DataZoomOpts(
                    is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                    ),
                # opts.DataZoomOpts(
                #   is_show=True, type_="slider",orient= "vertical",yaxis_index=[0, 0,1],range_start= -10,range_end=110
                #   ),
                ],
        tooltip_opts=opts.TooltipOpts(
            trigger="axis",
            axis_pointer_type="cross",
            background_color="rgba(245, 245, 245, 0.8)",
            border_width=1,
            border_color="#ccc",
            textstyle_opts=opts.TextStyleOpts(color="#000"),
            ),
        axispointer_opts=opts.AxisPointerOpts(
            is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
            ),
    )
)

line = (
    Line()
    .add_xaxis(df['交易日'].tolist())
    .add_yaxis(
        '组合净值',
        df['组合净值'].tolist(),
        yaxis_index=2,
        label_opts=opts.LabelOpts(is_show=False),
        color='#88c4e8')
    .add_yaxis(
        '基准1净值',
        df['基准1净值'].tolist(),
        yaxis_index=2,
        label_opts=opts.LabelOpts(is_show=False),
        color='#ea9c9d')
    .add_yaxis(
        '基准2净值',
        df['基准2净值'].tolist(),
        yaxis_index=2,
        label_opts=opts.LabelOpts(is_show=False),
        color='#93cc82')

)

bar.overlap(line)
grid = Grid()
grid.add(bar, opts.GridOpts(pos_left="5%", pos_right="20%"), is_control_axis_index=True)
grid.render("模拟组合历史数据1.html")

'c:\\Users\\siruo\\Downloads\\模拟组合历史回测\\模拟组合历史数据1.html'

In [26]:
bar = (
    Bar()
    .add_xaxis(df['交易日'].tolist())
    .add_yaxis(
        '相对基准1超额',
        df['基准1超额收益率'].tolist(),
        yaxis_index=0,
        label_opts=opts.LabelOpts(is_show=False),
        itemstyle_opts=opts.ItemStyleOpts(color='#F5F5DC',opacity=0.3),
        # color='#F5F5DC'
        )
    .add_yaxis(
        '相对基准2超额',
        df['基准2超额收益率'].tolist(),
        yaxis_index=0,
        label_opts=opts.LabelOpts(is_show=False),
        itemstyle_opts=opts.ItemStyleOpts(color='#EEE8AA',opacity=0.3),
        # color='#EEE8AA'
        )
    .extend_axis(
        yaxis=opts.AxisOpts(
            type_="value",
            name="收益",
            min_=0.9,
            max_=1.9,
            interval=0.1,
            position="left",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="#675bba")
            ),
            #axislabel_opts=opts.LabelOpts(formatter="{value} °C"),
            splitline_opts=opts.SplitLineOpts(
                is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=1)
            ),
        )
    )
    .set_global_opts(
        yaxis_opts=opts.AxisOpts(
            name="超额",
            min_=0,
            max_=1,
            interval=0.1,
            position="right",
            #offset=80,
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color='#88c4e8')
            ),
            #axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
            axislabel_opts=opts.LabelOpts(formatter=JsCode("function (value) {return value * 100 + '%';}")),
        ),
        title_opts=opts.TitleOpts(title="模拟组合历史回测"),
        #tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        datazoom_opts=[
            opts.DataZoomOpts(
                is_show=True, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
                ),
                opts.DataZoomOpts(
                    is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                    ),
                # opts.DataZoomOpts(
                #   is_show=True, type_="slider",orient= "vertical",yaxis_index=[0, 0,1],range_start= -10,range_end=110
                #   ),
                ],
        tooltip_opts=opts.TooltipOpts(
            trigger="axis",
            axis_pointer_type="cross",
            background_color="rgba(245, 245, 245, 0.8)",
            border_width=1,
            border_color="#ccc",
            textstyle_opts=opts.TextStyleOpts(color="#000"),
            ),
        axispointer_opts=opts.AxisPointerOpts(
            is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
            ),
    )
)

line = (
    Line()
    .add_xaxis(df['交易日'].tolist())
    .add_yaxis(
        '组合净值',
        df['组合净值'].tolist(),
        yaxis_index=1,
        label_opts=opts.LabelOpts(is_show=False),
        color='#93cc82')
    .add_yaxis(
        '基准1净值',
        df['基准1净值'].tolist(),
        yaxis_index=1,
        label_opts=opts.LabelOpts(is_show=False),
        color="#d14a61")
    .add_yaxis(
        '基准2净值',
        df['基准2净值'].tolist(),
        yaxis_index=1,
        label_opts=opts.LabelOpts(is_show=False),
        color="#5793f3")

)

bar.overlap(line)
grid = Grid()
grid.add(bar, opts.GridOpts(pos_left="5%", pos_right="20%"), is_control_axis_index=True)
grid.render("模拟组合历史数据2.html")

'c:\\Users\\siruo\\Downloads\\模拟组合历史回测\\模拟组合历史数据2.html'